In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_10k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_2k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv


In [2]:
%time !uv pip install autogluon

Using Python 3.11.13 environment at: /usr
Resolved 229 packages in 5.93s
   Building seqeval==1.2.2
   Building seqeval==1.2.2
   Building nvidia-ml-py3==7.352.0
   Building seqeval==1.2.2
   Building nvidia-ml-py3==7.352.0
⠙ Preparing packages... (0/50)
   Building seqeval==1.2.2
   Building nvidia-ml-py3==7.352.0
⠙ Preparing packages... (0/50)
distlib    ------------------------------     0 B/458.05 KiB
   Building seqeval==1.2.2
   Building nvidia-ml-py3==7.352.0
⠙ Preparing packages... (0/50)
psutil     ------------------------------     0 B/271.47 KiB
distlib    ------------------------------     0 B/458.05 KiB
   Building seqeval==1.2.2
   Building nvidia-ml-py3==7.352.0
⠙ Preparing packages... (0/50)
fsspec     ------------------------------     0 B/189.08 KiB
psutil     ------------------------------     0 B/271.47 KiB
distlib    ------------------------------     0 B/458.05 KiB
   Building seqeval==1.2.2
   Building nvidia-ml-py3==7.352.0
⠙ Preparing packages... (0/50)
dill   

In [3]:
#!pip uninstall pyarrow -y
#!pip install pyarrow==17.0.0

In [4]:
#!uv pip install -U autogluon > /dev/null
#!uv pip install autogluon > /dev/null

In [5]:
import autogluon.tabular as ag_tab
import pandas as pd
import pyarrow
import ray
from autogluon.tabular import TabularPredictor

print(f"Ray version: {ray.__version__}")
print(f"PyArrow version: {pyarrow.__version__}")
print("AutoGluon version:", ag_tab.__version__)

Ray version: 2.44.1
PyArrow version: 19.0.1
AutoGluon version: 1.4.0


In [6]:
from autogluon.tabular import TabularDataset, TabularPredictor
from pathlib import Path
import numpy as np # linear algebra
import pandas as pd 

In [7]:
path = Path('/kaggle/input/playground-series-s5e10/')
path

PosixPath('/kaggle/input/playground-series-s5e10')

In [8]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')
og_df_a = pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv')

In [9]:
train_df.shape,og_df_a.shape

((517754, 13), (100000, 13))

In [10]:
new_train_df = pd.concat([train_df, og_df_a], ignore_index=True)
new_train_df

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
617749,rural,2,0.61,60,dim,foggy,False,True,evening,False,False,1,0.54
617750,rural,2,0.52,35,dim,foggy,True,True,afternoon,False,False,1,0.18
617751,rural,2,0.08,70,daylight,clear,False,False,evening,True,False,1,0.20
617752,rural,1,0.83,70,dim,foggy,False,True,morning,False,True,0,0.58


In [11]:
og_df_a

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,rural,2,0.29,70,night,rainy,False,True,evening,False,False,1,0.64
1,highway,1,0.34,25,dim,clear,False,False,morning,False,False,3,0.27
2,rural,2,0.76,70,night,foggy,True,False,evening,True,True,1,0.76
3,rural,3,0.37,70,night,foggy,True,False,morning,False,True,0,0.60
4,highway,3,0.39,45,dim,rainy,False,True,morning,False,False,0,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,rural,2,0.61,60,dim,foggy,False,True,evening,False,False,1,0.54
99996,rural,2,0.52,35,dim,foggy,True,True,afternoon,False,False,1,0.18
99997,rural,2,0.08,70,daylight,clear,False,False,evening,True,False,1,0.20
99998,rural,1,0.83,70,dim,foggy,False,True,morning,False,True,0,0.58


In [12]:
train_df

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
id,,,,,,,,,,,,,
0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
517749,highway,4,0.10,70,daylight,foggy,True,True,afternoon,False,False,2,0.32
517750,rural,4,0.47,35,daylight,rainy,True,True,morning,False,False,1,0.26
517751,urban,4,0.62,25,daylight,foggy,False,False,afternoon,False,True,0,0.19


In [13]:
TabularPredictor.fit??

In [14]:
target = 'accident_risk'
eval_metric = 'rmse'
#train_data = train
train_data = new_train_df
Time_limit = 3600*1
#Time_limit = 900
problem_type='regression'

In [15]:
%%time
predictor = TabularPredictor(
    label=target, 
    eval_metric=eval_metric,
    verbosity=1,
    problem_type=problem_type
).fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN'], 
    time_limit=Time_limit,
    #dynamic_stacking=False,  # ← ADD THIS
    #ag_args_fit={'num_gpus': 1}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20251112_034514"
2025-11-12 03:45:16,502	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(_dystack pid=273) 	Warning: Model is expected to require 177.1s to train, which exceeds the maximum time limit of 2.6s, skipping model...
(_dystack pid=273) 	Warning: Reducing model 'n_estimators' from 300 -> 48 due to low time. Expected time usage reduced from 600.9s -> 97.2s...
2025-11-12 04:30:25,539	ERROR worker.py:420 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-11-12 04:30:25,542	ERROR worker.py:420 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-11-12 04:30:25,543	ERROR worker.py:420 -- Unhandled erro

CPU times: user 1h 2min 26s, sys: 15.1 s, total: 1h 2min 41s
Wall time: 1h 3s


In [16]:
%%time
predictor.leaderboard()

CPU times: user 5.16 ms, sys: 997 µs, total: 6.15 ms
Wall time: 5.69 ms


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.055931,root_mean_squared_error,230.104312,2370.738346,0.010285,0.909314,3,True,11
1,LightGBM_BAG_L2,-0.055988,root_mean_squared_error,214.279646,1760.492464,12.963799,80.267466,2,True,9
2,WeightedEnsemble_L2,-0.055999,root_mean_squared_error,69.017882,1345.602833,0.011820,0.747273,2,True,7
3,LightGBM_BAG_L1,-0.056032,root_mean_squared_error,29.032545,102.448495,29.032545,102.448495,1,True,2
4,LightGBMXT_BAG_L2,-0.056082,root_mean_squared_error,257.271747,1835.406716,55.955899,155.181718,2,True,8
5,CatBoost_BAG_L1,-0.056104,root_mean_squared_error,1.434263,892.135426,1.434263,892.135426,1,True,4
6,RandomForestMSE_BAG_L2,-0.056161,root_mean_squared_error,217.130228,2289.561565,15.814380,609.336567,2,True,10
7,LightGBMXT_BAG_L1,-0.056236,root_mean_squared_error,129.391488,319.298881,129.391488,319.298881,1,True,1
8,ExtraTreesMSE_BAG_L1,-0.056335,root_mean_squared_error,19.559215,166.508377,19.559215,166.508377,1,True,5
9,RandomForestMSE_BAG_L1,-0.056501,root_mean_squared_error,18.980039,183.763262,18.980039,183.763262,1,True,3


In [17]:
import autogluon.tabular as ag_tab
import pandas as pd
import pyarrow
import ray
from autogluon.tabular import TabularPredictor

print(f"Ray version: {ray.__version__}")
print(f"PyArrow version: {pyarrow.__version__}")
print("AutoGluon version:", ag_tab.__version__)

Ray version: 2.44.1
PyArrow version: 19.0.1
AutoGluon version: 1.4.0


In [18]:
%%time
autogluon_preds = predictor.predict(test_df)

CPU times: user 7min 28s, sys: 3.71 s, total: 7min 31s
Wall time: 2min 5s


In [19]:
autogluon_preds

id
517754    0.298086
517755    0.119645
517756    0.182452
517757    0.343276
517758    0.384226
            ...   
690334    0.106808
690335    0.516529
690336    0.246711
690337    0.128544
690338    0.489274
Name: accident_risk, Length: 172585, dtype: float32

In [20]:
!rm submission.csv
sub_df['accident_risk'] = autogluon_preds.values
sub_df.to_csv('submission.csv', index=False)
sub_df.head()

rm: cannot remove 'submission.csv': No such file or directory


,id,accident_risk
0,517754,0.298086
1,517755,0.119645
2,517756,0.182452
3,517757,0.343276
4,517758,0.384226
